In [0]:
!pip install faiss-cpu==1.7.4 tiktoken openai
!pip install transformers InstructorEmbedding
!pip install sentence-transformers
!pip install typing-extensions
!python -m pip install --upgrade pip

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.1 MB/s eta 0:00:00


In [0]:
%pip install --force-reinstall langchain==0.0.251

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.2/375.2 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [0]:
dbutils.library.restartPython()

#Define imports, global variables and functions

In [5]:
from pyspark.sql.functions import col,lit
from pyspark.sql.types import *
import pandas as pd
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import os
import openai
import logging as logger
from langchain import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings

ModuleNotFoundError: No module named 'tiktoken'

In [0]:
if 'config' not in locals():
  config = {}

# Default Instructor Model
config["vector_store_path"] = "/dbfs/tmp/ars_embeddings_1"
config["data_path"] = "s3://ds-databricks-playground/prarthana/ars/data_pq/"
config["search_kwargs"] = {'k': 30, 'score_threshold': 0.9}
config["embedding_model_name"] = "sentence-transformers/all-mpnet-base-v2"
config["embedding_model_kwargs"] = {'device': 'cpu'}
config["embedding_model_encode_kwargs"] = {'normalize_embeddings': False}

In [0]:
def get_combined(x, columns):
  row_str = ""
  for col_name in columns:
      col_value = f"x.{col_name}"
      if row_str == "":
        row_str += f"{col_name}: {eval(col_value)}"
      else:
        row_str += f"; {col_name}: {eval(col_value)}"
  return row_str

def get_decombined(x):
    return [i.split(": ")[1].replace("None", "null") for i in x.split("; ")]

def get_column_names(x):
    return [i.split(": ")[0] for i in x.split("; ")]

# Trying with langchain's embeddings - Private Dummy Data

In [0]:
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import os
import openai
os.environ["OPENAI_API_KEY"] = "sk-"
openai.apikey = os.getenv("OPENAI_API_KEY")
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 5000  # the maximum for text-embedding-ada-002 is 8191

In [0]:
prompt_template = """
You are an intelligent recommendation bot who analyses and recommends users simillir to given users. Answer best of your knowledge.
Following is the user information about his job:

Context:
{context}

Create table for 20 users similar to below data(excluding below data), take a while to think:
{users}
"""

In [0]:
dummy_df = spark.read.csv("/FileStore/tmp/temp.csv", header=True)
display(dummy_df)
dummy_df.columns

id,apple_height,apple_weight,apple_tree_height_meter,apple_tree_age_years,apple_type
1,5.1,3.5,1.4,0.2,apple_ps
2,4.9,3.0,1.4,0.2,apple_ps
3,4.7,3.2,1.3,0.2,apple_ps
4,4.6,3.1,1.5,0.2,apple_ps
5,5.0,3.6,1.4,0.2,apple_ps
6,5.4,3.9,1.7,0.4,apple_ps
7,4.6,3.4,1.4,0.3,apple_ps
8,5.0,3.4,1.5,0.2,apple_ps
9,4.4,2.9,1.4,0.2,apple_ps
10,4.9,3.1,1.5,0.1,apple_ps


Out[61]: ['id',
 'apple_height',
 'apple_weight',
 'apple_tree_height_meter',
 'apple_tree_age_years',
 'apple_type']

In [0]:
encoding = tiktoken.get_encoding(embedding_encoding)
def get_combined(x):
  return str("apple_height: " + str(x.apple_height) + 
    "; apple_weight: " + str(x.apple_weight) +  "; apple_tree_height_meters: " + str(x.apple_tree_height_meter) +
    "; apple_tree_age_years: " + str(x.apple_tree_age_years))

def get_decombined(x):
    return [i.split(": ")[1] for i in x.split(";")]

In [0]:
# identify embedding model that will generate embedding vectors
# Convert to prompt
from langchain.embeddings import OpenAIEmbeddings
# from langchain import Chroma
from langchain import FAISS
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

vector_store = FAISS.from_texts(
  embedding=embeddings, 
  texts=list(dummy_df.drop("apple_type").rdd.map(lambda x: get_combined(x)).collect())
  )

In [0]:
vsr = vector_store.as_retriever(search_kwargs={'k': 30, 'score_threshold': 0.99})

In [0]:
rows_list = dummy_df.filter(F.col("apple_type")=="apple_ps").limit(10).rdd.map(lambda x: get_combined(x)).collect()
relevant_rows = vsr.get_relevant_documents("\n".join(rows_list))
rows_list, relevant_rows

Out[74]: (['apple_height: 5.1; apple_weight: 3.5; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.2',
  'apple_height: 4.9; apple_weight: 3.0; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.2',
  'apple_height: 4.7; apple_weight: 3.2; apple_tree_height_meters: 1.3; apple_tree_age_years: 0.2',
  'apple_height: 4.6; apple_weight: 3.1; apple_tree_height_meters: 1.5; apple_tree_age_years: 0.2',
  'apple_height: 5.0; apple_weight: 3.6; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.2',
  'apple_height: 5.4; apple_weight: 3.9; apple_tree_height_meters: 1.7; apple_tree_age_years: 0.4',
  'apple_height: 4.6; apple_weight: 3.4; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.3',
  'apple_height: 5.0; apple_weight: 3.4; apple_tree_height_meters: 1.5; apple_tree_age_years: 0.2',
  'apple_height: 4.4; apple_weight: 2.9; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.2',
  'apple_height: 4.9; apple_weight: 3.1; apple_tree_height_meters: 1.5; apple_tree_age_year

In [0]:
converted_rows = [get_decombined(row.page_content) for row in relevant_rows]
relevant_df = spark.createDataFrame(converted_rows, schema=dummy_df.drop("id", "apple_type").schema)

relevant_df_joined = relevant_df.join(dummy_df, how="inner", on=dummy_df.drop("apple_type", "id").columns)
relevant_df_joined.show()

+------------+------------+-----------------------+--------------------+---+----------+
|apple_height|apple_weight|apple_tree_height_meter|apple_tree_age_years| id|apple_type|
+------------+------------+-----------------------+--------------------+---+----------+
|         5.0|         3.3|                    1.4|                 0.2| 50|  apple_ps|
|         5.0|         3.2|                    1.2|                 0.2| 36|  apple_ps|
|         5.0|         3.5|                    1.3|                 0.3| 41|  apple_ps|
|         5.0|         3.4|                    1.5|                 0.2|  8|  apple_ps|
|         5.0|         2.0|                    3.5|                 1.0| 61|  apple_am|
|         5.0|         3.4|                    1.6|                 0.4| 27|  apple_ps|
|         5.0|         3.0|                    1.6|                 0.2| 26|  apple_ps|
|         5.0|         3.6|                    1.4|                 0.2|  5|  apple_ps|
|         5.0|         2.3|     

In [0]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.0)
prompt = PromptTemplate(
    input_variables=["users", "context"],
    template=prompt_template,
)


In [0]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run(users="\n".join(rows_list), context="\n".join([row.page_content for row in relevant_rows])))

apple_height: 5.4; apple_weight: 3.7; apple_tree_height_meters: 1.5; apple_tree_age_years: 0.2
apple_height: 4.8; apple_weight: 3.4; apple_tree_height_meters: 1.6; apple_tree_age_years: 0.2
apple_height: 4.8; apple_weight: 3.0; apple_tree_height_meters: 1.4; apple_tree_age_years: 0.1
apple_height: 4.3; apple_weight: 3.0; apple_tree_height_meters: 1.1; apple_tree_age_years: 0.1
apple_height: 5.8; apple_weight: 2.7; apple_tree_height_meters: 3.9; apple_tree_age_years: 1.2
apple_height: 5.7; apple_weight: 4.4; apple_tree_height_meters: 1.5; apple_tree_age_years: 0.4
apple_height: 5.4; apple_


#Tesmp

In [6]:
%pip install --force-reinstall zipp==3.15 langchain gpt4all pypdf==3.13.0 chromadb sentence_transformers InstructorEmbedding fastapi redis pydantic==1.10.13

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 816 kB 2.2 MB/s            
     |████████████████████████████████| 5.8 MB 12.7 MB/s            
     |████████████████████████████████| 256 kB 11.7 MB/s            
     |████████████████████████████████| 521 kB 18.8 MB/s            
     |████████████████████████████████| 149 kB 29.4 MB/s            
     |████████████████████████████████| 92 kB 30.7 MB/s            
     |████████████████████████████████| 250 kB 29.8 MB/s            
     |████████████████████████████████| 2.9 MB 18.9 MB/s            
     |████████████████████████████████| 197 kB 22.6 MB/s            
     |████████████████████████████████| 62 kB 25.4 MB/s            
     |████████████████████████████████| 1.7 MB 40.3 MB/s            
     |████████████████████████████████| 2.1 MB 24.2 MB/s            
     |████████████████████████████████| 246 kB 3.9 MB/s            
     |████████████████████████████

     |████████████████████████████████| 55 kB 40.9 MB/s            
     |████████████████████████████████| 60 kB 32.0 MB/s            
     |████████████████████████████████| 53 kB 20.5 MB/s            
     |████████████████████████████████| 49 kB 33.3 MB/s            
     |████████████████████████████████| 170 kB 41.0 MB/s            
     |████████████████████████████████| 186 kB 33.2 MB/s            
     |████████████████████████████████| 163 kB 29.0 MB/s            
     |████████████████████████████████| 247 kB 21.4 MB/s            
     |████████████████████████████████| 151 kB 49.1 MB/s            
     |████████████████████████████████| 121 kB 34.5 MB/s            
     |████████████████████████████████| 58 kB 30.6 MB/s            
     |████████████████████████████████| 85 kB 30.0 MB/s            
     |████████████████████████████████| 394 kB 27.8 MB/s            
     |████████████████████████████████| 5.7 MB 19.0 MB/s            
     |████████████████████████████████| 

     |████████████████████████████████| 86 kB 36.1 MB/s            
     |████████████████████████████████| 536 kB 32.4 MB/s            
     |████████████████████████████████| 84 kB 40.0 MB/s            
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53723 sha256=058cc59b34d16d5cb82de1dfb0e358b297d4f42499d777a44acaa115b5e6c8a0
  Stored in directory: /private/var/folders/g1/wqd2jbyj7v5cdjc49p0f_1200000gq/T/pip-ephem-wheel-cache-6w29_ld7/wheels/f7/02/64/d541eac67ec459309d1fb19e727f58ecf7ffb4a8bf42d4cfe5
  Created wheel for greenlet: filename=greenlet-3.0.3-cp39-cp39-macosx_10_9_x86_64.whl size=213476 sha256=49f24dc76c699df1b0709ec510669a18754c80407a7895c9b3b94561385ee9cf
  Stored in directory: /private/var/folders/g1/wqd2jbyj7v5cdjc49p0f_1200000gq/T/pip-ephem-wheel-cache-6w29_ld7/wheels/4e/b7/50/fef77b0dafc995530497e09a5123f00b1e68c9d305e9cf37e5
Successfully built pypika greenlet
  Attempting uninstall: zipp
    Found existing installation: zipp 3.8.0
    U

    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: rsa
    Found existing installation: rsa 4.7.2
    Uninstalling rsa-4.7.2:
      Successfully uninstalled rsa-4.7.2
  Attempting uninstall: pyasn1-modules
    Found existing installation: pyasn1-modules 0.2.8
    Uninstalling pyasn1-modules-0.2.8:
      Successfully uninstalled pyasn1-modules-0.2.8
  Attempting uninstall: opentelemetry-util-http
    Found existing installation: opentelemetry-util-http 0.44b0
    Uninstalling opentelemetry-util-http-0.44b0:
      Successfully uninstalled opentelemetry-util-http-0.44b0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.44b0
    Uninstalling opentelemetry-semantic-conventions-0.44b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.44b0
  Attempting uninstall: opentelemetry-proto
    Found existing installa

      Successfully uninstalled monotonic-1.6
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.26
    Uninstalling langchain-core-0.1.26:
      Successfully uninstalled langchain-core-0.1.26
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successfully uninstalled joblib-1.2.0
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2
  Attempting uninstall: httptools
    Found existing installation: httptools 0.6.0
    Uninstalling httptools-0.6.0:
      Successfully uninstalled httptools-0.6.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.0
    Uninstalling grpcio-1.59.0:
      Successfully uninstalled grpcio-1.59.0
  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.62.0
    Uninstalling googleapis-com

You should consider upgrading via the '/usr/local/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
data_path = "/Users/pshah1/Downloads/look_alike/aes_data"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("customer_look_alike_modelling").getOrCreate()
df = spark.read.option("header", "true").parquet(data_path)
df.limit(5).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 09:03:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/27 09:03:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/27 09:03:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+----------------------+-----------+---------------------+---------------+------------+---------------+-------------+--------+--------------+-----------------+--------------------------+----------------------+------------------------+------------------------+-----------+-----------+---------------------+-----------------+-------------+------+-----------------+-----------------+------------+----------------------+-----+
|infogroup_id|mapped_contact_id_cont|gender_cont|management_level_cont|job_titles_cont|primary_cont|infogroup_id_pl|place_type_pl|state_pl|       city_pl|contacts_count_pl|location_employee_count_pl|primary_sic_code_id_pl|location_sales_volume_pl|primary_naics_code_id_pl|b_abinumber|e_contactid|e_executivesourcecode|b_fulfillmentflag|b_countrycode|target|bus_abinumber_b2c|bus_contactid_b2c|cons_age_b2c|cons_maritalstatus_b2c|count|
+------------+----------------------+-----------+---------------------+---------------+------------+---------------+------------

In [3]:

# rename to cols_to_convert
# rows_to_convert = "job_titles_cont,city_pl".split(",")
rows_to_convert = df.columns
rows_to_convert.remove("infogroup_id")
rows_to_convert.remove("count")
rows_to_convert.remove("infogroup_id_pl")
rows_to_convert.remove("e_contactid")
rows_to_convert.remove("target")
rows_to_convert

['mapped_contact_id_cont',
 'gender_cont',
 'management_level_cont',
 'job_titles_cont',
 'primary_cont',
 'place_type_pl',
 'state_pl',
 'city_pl',
 'contacts_count_pl',
 'location_employee_count_pl',
 'primary_sic_code_id_pl',
 'location_sales_volume_pl',
 'primary_naics_code_id_pl',
 'b_abinumber',
 'e_executivesourcecode',
 'b_fulfillmentflag',
 'b_countrycode',
 'bus_abinumber_b2c',
 'bus_contactid_b2c',
 'cons_age_b2c',
 'cons_maritalstatus_b2c']

In [4]:
",".join(rows_to_convert)

'mapped_contact_id_cont,gender_cont,management_level_cont,job_titles_cont,primary_cont,place_type_pl,state_pl,city_pl,contacts_count_pl,location_employee_count_pl,primary_sic_code_id_pl,location_sales_volume_pl,primary_naics_code_id_pl,b_abinumber,e_executivesourcecode,b_fulfillmentflag,b_countrycode,bus_abinumber_b2c,bus_contactid_b2c,cons_age_b2c,cons_maritalstatus_b2c'

In [5]:
df = df.filter(df.job_titles_cont.isNotNull())

In [6]:
df.count()

80202

In [7]:
from src.utils.functions import get_row_as_text, hf_embeddings, get_ars_retrieved_df
train_df = get_row_as_text(df.limit(4000), rows_to_convert)

In [8]:
train_df.select("row_as_text").show(2, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|row_as_text                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
train_df.printSchema()

root
 |-- infogroup_id: string (nullable = true)
 |-- mapped_contact_id_cont: string (nullable = true)
 |-- gender_cont: string (nullable = true)
 |-- management_level_cont: string (nullable = true)
 |-- job_titles_cont: string (nullable = true)
 |-- primary_cont: string (nullable = true)
 |-- infogroup_id_pl: string (nullable = true)
 |-- place_type_pl: string (nullable = true)
 |-- state_pl: string (nullable = true)
 |-- city_pl: string (nullable = true)
 |-- contacts_count_pl: string (nullable = true)
 |-- location_employee_count_pl: string (nullable = true)
 |-- primary_sic_code_id_pl: string (nullable = true)
 |-- location_sales_volume_pl: string (nullable = true)
 |-- primary_naics_code_id_pl: string (nullable = true)
 |-- b_abinumber: string (nullable = true)
 |-- e_contactid: string (nullable = true)
 |-- e_executivesourcecode: string (nullable = true)
 |-- b_fulfillmentflag: string (nullable = true)
 |-- b_countrycode: string (nullable = true)
 |-- target: integer (nullable = 

Unique columns: infogroup_id, e_contactid

In [10]:
step = 1000
k = 4000

texts_list = train_df.limit(4000).rdd.collect()
# texts_list = [x[0] for x in texts_label_list]
# texts_list
texts_list[0]

Row(infogroup_id='706609709', mapped_contact_id_cont='000070360924', gender_cont='M', management_level_cont='executive', job_titles_cont='President', primary_cont='true', infogroup_id_pl='706609709', place_type_pl='independent', state_pl='TX', city_pl='El Paso', contacts_count_pl='1', location_employee_count_pl='4', primary_sic_code_id_pl='176109', location_sales_volume_pl='446000', primary_naics_code_id_pl='23816014', b_abinumber=None, e_contactid=None, e_executivesourcecode=None, b_fulfillmentflag=None, b_countrycode=None, target=0, bus_abinumber_b2c=None, bus_contactid_b2c=None, cons_age_b2c=None, cons_maritalstatus_b2c=None, count=1, row_as_text='mapped_contact_id_cont: 000070360924; gender_cont: M; management_level_cont: executive; job_titles_cont: President; primary_cont: true; place_type_pl: independent; state_pl: TX; city_pl: El Paso; contacts_count_pl: 1; location_employee_count_pl: 4; primary_sic_code_id_pl: 176109; location_sales_volume_pl: 446000; primary_naics_code_id_pl: 

In [12]:
import os

# Specify the path of the directory you want to create
db_dir = "/Users/pshah1/ps/projects/look_alike_modelling/src/resources/ars_embeddings/embeddings_all_14"

# Create the directory
os.makedirs(db_dir, exist_ok=True)

In [13]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb
import os

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [17]:
from langchain.vectorstores import Chroma
vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
for i in range(0, len(texts_list), step):
    texts = [x.row_as_text for x in texts_list[i:i+step]]
    metadata = [{"infogroup_id": str(x.infogroup_id), "mapped_contact_id_cont": str(x.mapped_contact_id_cont)} for x in texts_list[i:i+step]]
    vdb.add_texts(texts, metadata)
    vdb.persist()

In [15]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb

db_dir = "/Users/pshah1/ps/projects/look_alike_modelling/src/resources/ars_embeddings/embeddings_all_4"


client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [18]:
vdb._collection.count()

4000

In [38]:
vdb.embeddings.embed_query("hello")

[0.03063981980085373,
 -0.006230222526937723,
 -0.002121522556990385,
 0.013879158534109592,
 0.026486849412322044,
 0.004065817687660456,
 -0.003871251828968525,
 0.035269834101200104,
 0.011488324962556362,
 -0.003806537948548794,
 0.01323637180030346,
 -0.011885665357112885,
 -0.0001603731361683458,
 0.02157067507505417,
 0.03014964610338211,
 -0.10559132695198059,
 0.03541700914502144,
 0.006596209481358528,
 -0.055303871631622314,
 -0.000226780422963202,
 -0.007831414230167866,
 -0.004739547614008188,
 -2.192328975070268e-05,
 0.037050411105155945,
 0.02242698334157467,
 0.004295805934816599,
 -0.001075013424269855,
 0.007416184525936842,
 0.011647200211882591,
 0.019009582698345184,
 -0.011946482583880424,
 0.0021739050280302763,
 0.026941843330860138,
 0.016080964356660843,
 2.160506255677319e-06,
 -0.008001399226486683,
 -0.013405386358499527,
 0.016731996089220047,
 -0.009559343568980694,
 -0.022413142025470734,
 -0.00016311132640112191,
 -0.0004943842650391161,
 -0.0090955542

In [0]:
# ! zip -r embeddings_ars_01.zip /dbfs/tmp/ps/ars/embeddings_01

  adding: dbfs/tmp/ps/ars/embeddings_01/ (stored 0%)
  adding: dbfs/tmp/ps/ars/embeddings_01/chroma-collections.parquet (deflated 54%)
  adding: dbfs/tmp/ps/ars/embeddings_01/chroma-embeddings.parquet (deflated 20%)
  adding: dbfs/tmp/ps/ars/embeddings_01/index/ (stored 0%)
  adding: dbfs/tmp/ps/ars/embeddings_01/index/id_to_uuid_e4b4c1d9-13cf-4a78-bd9c-960a4a3b2962.pkl (deflated 37%)
  adding: dbfs/tmp/ps/ars/embeddings_01/index/index_e4b4c1d9-13cf-4a78-bd9c-960a4a3b2962.bin (deflated 10%)
  adding: dbfs/tmp/ps/ars/embeddings_01/index/index_metadata_e4b4c1d9-13cf-4a78-bd9c-960a4a3b2962.pkl (deflated 14%)
  adding: dbfs/tmp/ps/ars/embeddings_01/index/uuid_to_id_e4b4c1d9-13cf-4a78-bd9c-960a4a3b2962.pkl (deflated 41%)


In [0]:
# !ls

azure  embeddings_ars_01.zip  hadoop_accessed_config.lst  preload_class.lst
conf   eventlogs	      logs


In [0]:
# !cp embeddings_ars_01.zip /dbfs/tmp/ps/

In [0]:
# dbutils.fs.cp("/tmp/ps/embeddings_ars_01.zip", "s3://ds-databricks-playground/prarthana_output/ars_embeddings_10k/")

True

In [29]:
from pyspark.sql import functions as F
val_df = train_df.filter(F.col("job_titles_cont") == F.lit("Owner")).filter(F.col("city_pl") == "Houston")
val_df.show()
val_df.count()


+------------+----------------------+-----------+---------------------+---------------+------------+---------------+-------------+--------+-------+-----------------+--------------------------+----------------------+------------------------+------------------------+-----------+------------+---------------------+-----------------+-------------+------+-----------------+-----------------+------------+----------------------+-----+--------------------+
|infogroup_id|mapped_contact_id_cont|gender_cont|management_level_cont|job_titles_cont|primary_cont|infogroup_id_pl|place_type_pl|state_pl|city_pl|contacts_count_pl|location_employee_count_pl|primary_sic_code_id_pl|location_sales_volume_pl|primary_naics_code_id_pl|b_abinumber| e_contactid|e_executivesourcecode|b_fulfillmentflag|b_countrycode|target|bus_abinumber_b2c|bus_contactid_b2c|cons_age_b2c|cons_maritalstatus_b2c|count|         row_as_text|
+------------+----------------------+-----------+---------------------+---------------+-----------

37

In [28]:
val_df.limit(10).toPandas().to_csv("/Users/pshah1/Downloads/owner_houston.csv")

In [93]:
for i in texts_list:
    if i.city_pl == "Miami":
        print(str(i.city_pl))

Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
Miami
